In [177]:
import numpy as np
import matplotlib.pyplot as plt

Use https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz MNIST
dataset for this question and select two digits - 0 and 1. Label them as -1 and
1.


Divide the train set into train and val set. Keep 1000 samples from each
class for val.

In [ ]:
with np.load("C:\Shared_archcraft\SML\Assignment4\mnist.npz") as data:
    x_train, y_train = data['x_train'], data['y_train']
    x_test, y_test = data['x_test'], data['y_test']

train_mask = np.isin(y_train, [0])
x_train_0 = x_train[train_mask]
y_train_0 = [-1]*len(x_train_0)

train_mask = np.isin(y_train, [1])
x_train_1 = x_train[train_mask]
y_train_1 = y_train[train_mask]

train_mask = np.isin(y_train, [0, 1])
x_train_01 = x_train[train_mask]
y_train_01 = y_train[train_mask]

print(len(y_train_0))
print(len(y_train_1))


x_test_1 = x_train_1[:1000]
y_test_1 = y_train_1[:1000]
x_test_0 = x_train_0[:1000]
y_test_0 = y_train_0[:1000]

x_train_0 = x_train_0[1000:]
y_train_0 = y_train_0[1000:]
x_train_1 = x_train_1[1000:]
y_train_1 = y_train_1[1000:]

x_real_train = np.concatenate([x_train_0, x_train_1], axis=0)
y_real_train = np.concatenate([y_train_0, y_train_1], axis=0)
x_real_test = np.concatenate([x_test_0, x_test_1], axis=0)
y_real_test = np.concatenate([y_test_0, y_test_1], axis=0)

# Reshape x_real_train
# x_real_train = x_real_train.reshape(-1, 28, 28)
print(y_real_train.shape)
print(x_real_train.shape)



In [179]:
def pca(x):
    X_mean = np.mean(x, axis=0)
    X_centered = x - X_mean
    X_centered_2d = X_centered.reshape(X_centered.shape[0], -1)  # Ensure X_centered is 2D
    cov_matrix = np.cov(X_centered_2d, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

    p = 5
    top_indices = np.argsort(eigenvalues)[::-1][:p]
    pca_matrix = eigenvectors[:, top_indices]
    
    x_reduced = np.dot(X_centered_2d, pca_matrix)
    
    # print(x_reduced.shape)
    return x_reduced

# pca(x_real_train)


In [180]:
def calculate_gini(labels):
    n_labels = len(labels)
    if n_labels == 0:
        return 0
    # Shift labels to ensure all values are non-negative
    shifted_labels = labels + 1  # Shift -1 to 0 and 1 to 2
    counts = np.bincount(shifted_labels, minlength=3)
    print(counts)  # Ensure array has at least 3 elements
    probabilities = counts / n_labels
    gini = 1 - np.sum(probabilities ** 2)
    return gini

def find_best_split(feature, labels, weight):
    sorted_indices = np.argsort(feature)
    sorted_weights = np.array(weight)[sorted_indices]
    print(sorted_indices)
    best_gini = float('inf')
    best_split_value = None
    for i in range(len(feature) - 1):
        split_value = 0.5 * (feature[sorted_indices[i]] + feature[sorted_indices[i + 1]])
        left_labels = labels[sorted_indices[:i + 1]]
        right_labels = labels[sorted_indices[i + 1:]]
        gini = (len(left_labels) * calculate_gini(left_labels) + len(right_labels) * calculate_gini(right_labels)) / len(labels)
        if gini < best_gini:
            best_gini = gini
            best_split_value = split_value
    return best_gini, best_split_value
def grow_decision_tree(features, labels, weight):
    n_features = features.shape[1]
    best_split_dim = None
    best_split_value = None
    best_gini= float('inf')
    
    print("dimensions ::", n_features)
    for dim in range(n_features):
        gini, split_value = find_best_split(features[:,dim], labels, weight)
        if gini < best_gini:
            best_gini = gini
            best_split_dim = dim
            best_split_value = split_value
    
    left_indices = features[:, best_split_dim] <= best_split_value
    right_indices = ~left_indices
    left_labels = labels[left_indices]
    right_labels = labels[right_indices]
    left_features = features[left_indices]
    right_features = features[right_indices]
    gini_left = calculate_gini(left_labels)
    gini_right = calculate_gini(right_labels)

    node = {
        'split_dim': best_split_dim,
        'split_value': best_split_value,
        'left': {'class': np.argmax(np.bincount(left_labels + 1)) - 1},
        'gini_left': gini_left,
        'gini_right': gini_right,
        'right': {'class': np.argmax(np.bincount(right_labels + 1)) - 1}
    }
    return node

def classify_sample(sample, node):
    # print(sample)
    if 'class' in node:
        return node['class']
    elif sample[node['split_dim']] <= node['split_value']:
        return classify_sample(sample, node['left'])
    else:
        return classify_sample(sample, node['right'])


def classify_samples(samples, node):
    predictions = []
    # print(samples.shape)
    for sample in samples:
        class_prediction = classify_sample(sample, node)
        predictions.append(class_prediction)
    return predictions
def find_Wrong_classfication(node, x_real_train, y_real_train, weight):
    a = 0
    total = 0
    print(x_real_train.shape)
    x = pca(x_real_train)
    for i in range(len(x_real_train)):
        if classify_sample(x[i], node) != y_real_train[i]:
            a+=weight[i]
        total += weight[i]
    return a/total
        
    # if gini_left >= gini_right:
    #     node = {
    #         'split_dim': best_split_dim,
    #         'split_value': best_split_value,
    #         'left': grow_decision_tree(left_features, left_labels, max_nodes-1),
    #         'gini_left': gini_left,
    #         'gini_right': gini_right,
    #         'right': {'class': np.argmax(np.bincount(right_labels))}
    #     }
    #     return node
    # else:
    #     node = {
    #         'split_dim': best_split_dim,
    #         'split_value': best_split_value,
    #         'left': {'class': np.argmax(np.bincount(left_labels))},
    #         'gini_left': gini_left,
    #         'gini_right': gini_right,
    #         'right': grow_decision_tree(right_features, right_labels, max_nodes-1)
    #     }
    #     return node

In [181]:
# weight = [1/len(x_real_train)]*len(x_real_train)
# print("HI",y_real_train.shape)
# # pca2(x_real_train)
# # pca(x_real_train)
# print(x_real_train.shape)
# pcaa = pca(x_real_train)
# node = grow_decision_tree(pcaa, y_real_train, weight)
# print(node)
# print(x_real_train.shape)



In [182]:

class DecisionStump:
    def __init__(self):
        self.idx = None
        self.iii = None

    def train(self, X, y, weights):
        num_samples = X.shape[0]  # Get the number of samples
        num_features = np.prod(X.shape[1:])  # Calculate the total number of features after flattening

        min_error = float('inf')

        # Flatten each image into a 1-dimensional array
        X_flat = X.reshape(num_samples, num_features)
        # print(X_flat)
        for idx in range(num_features):
            value = np.unique(X_flat[:, idx])#sort
            # print(value)
            for ii in value:
                pred = np.ones(num_samples)
                pred[X_flat[:, idx] < ii] = -1
                error = np.sum(weights[pred != y])

                if error < min_error:
                    min_error = error
                    self.idx = idx
                    self.iii = ii
                    

    def predict(self, X):
        num_samples = X.shape[0]
        num_features = np.prod(X.shape[1:])  # Calculate the total number of features after flattening

        # Flatten each image into a 1-dimensional array
        X_flat = X.reshape(num_samples, num_features)

        pred = np.ones(num_samples)
        # print("Dim",self.idx)
        pred[X_flat[:, self.idx] < self.iii] = -1
        return pred

stumps = []
alphas = []
accuracies = []
class AdaBoost:
    def __init__(self, num_stumps):
        self.num_stumps = num_stumps


    # def train(self, X, y):
    #     num_samples = X.shape[0]
    #     weights = ([1]*num_samples) / num_samples
    #     print(weights)

    #     for _ in range(self.num_stumps):
    #         st = DecisionStump()
    #         st.train(X, y, weights)
    #         error = np.sum(weights[st.predict(X) != y])
    #         alpha = np.log((1 - error) / error)
    #         stumps.append(st)
    #         alphas.append(alpha)
    #         weights = (weights *  np.exp(-0.5 * alpha * y * st.predict(X)))/np.sum(weights)

    def trainAndPredict(self, X, y):
        num_samples = X.shape[0]
        weights = np.array([1] * num_samples) / num_samples
        # print(weights)


        for i in range(self.num_stumps):
            print(i)
            st = DecisionStump()
            st.train(X, y, weights)
            error = np.sum(weights[st.predict(X) != y])
            alpha = 0.5 * np.log((1 - error) / error)
            # print(alpha)
            stumps.append(st)
            alphas.append(alpha)
            weights *= np.exp(-alpha * y * st.predict(X))
            weights /= np.sum(weights)
            pred = self.predict(X)
            accuracy = np.mean(pred == y)  # Compute accuracy
            accuracies.append(accuracy)
            # print("Error in Boost",i,":", 1 - accuracy)

        return accuracies

    def predict(self, X):
        num_samples = X.shape[0]
        pred = np.zeros(num_samples)
        for i in range(len(stumps)):
            pred += alphas[i] * stumps[i].predict(X)
        # pred = alphas[-1] * stumps[-1].predict(X)
        return np.sign(pred)

# Example usage





In [ ]:
pcaa = pca(x_real_train)
pcaa2 = pca(x_real_test)
adaboost = AdaBoost(num_stumps = 300)
# predictions = adaboost.predict(pcaa)
val_accuracies = adaboost.trainAndPredict(pcaa, y_real_train)

# val_accuracies = adaboost.trainAndPredict(pcaa2, y_real_test)

In [ ]:
print(val_accuracies)
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies)
plt.xlabel('Number of Trees')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy vs. Number of Trees')
plt.show()
n = 0
some = 0
predictions = np.zeros(len(x_real_test))

for i in range(len(val_accuracies)):
    if val_accuracies[i] > n:
        n = val_accuracies[i]
        some = i
for i in range(some + 1):
    # clf = stumps[i]
    # alpha = alphas[i]
    predictions += alphas[i] * stumps[i].predict(pcaa2)

    some -= 1
    if some < 0:
        break

predictions = np.sign(predictions)
# print(predictions)
accuracy = np.mean(predictions == y_real_test)
print("Accuracy for test is ::",accuracy)

In [ ]:
# Evaluate accuracy
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

# Example usage
predictions_train = adaboost.predict(pcaa)
accuracy_train = max(val_accuracies)
print("Training accuracy:", accuracy_train) 

predictions_test = adaboost.predict(pcaa2)
accuracy_test = accuracy(y_real_test, predictions_test)
print("Test accuracy:", accuracy_test)

Q2 Starts here

In [186]:
# NumOfStumps = 300
# residue = y_real_train.copy()
# residue = residue.astype(np.float64)
# X = pca(x_real_train)

NumOfStumps = 300
residue = y_real_train.copy()
residue = residue.astype(np.float64)
X = pca(x_real_train)

In [187]:
class DecisionStump:
    def __init__(self):
        self.idx = None
        self.iii = None
        self.thresh = None

    def train(self, X, y):
        num_samples = X.shape[0]  # Get the number of samples
        num_features = np.prod(X.shape[1:])  # Calculate the total number of features after flattening

        min_error = float('inf')
        # left = []
        # right = []

        # Flatten each image into a 1-dimensional array
        X_flat = X.reshape(num_samples, num_features)
        # print(X_flat)
        for idx in range(num_features):
            value = np.unique(X_flat[:, idx])#sort  
            # print(value)
            for ii in value:
                left_indices = X[:, idx] <= ii
                right_indices = X[:, idx] > ii
                # print("left",len(left_indices))
                
                left_mean = np.mean(y[left_indices])
                right_mean = np.mean(y[right_indices])
                ssr = np.sum((y[left_indices] - left_mean) ** 2) + np.sum((y[right_indices] - right_mean) ** 2)
            
            # Update best split if SSR is minimized
                if ssr < min_error:
                    min_error = ssr
                    self.iii = {'dim': idx, 'split': ii, 'left_mean': left_mean, 'right_mean': right_mean, "ssr": ssr}
                    self.thresh = ii

    def predict(self, X):
        num_samples = X.shape[0]
        num_features = np.prod(X.shape[1:])  # Calculate the total number of features after flattening

        # Flatten each image into a 1-dimensional array
        X_flat = X.reshape(num_samples, num_features)

        pred = np.ones(num_samples)
        # Access the 'dim' attribute from 'self.iii' instead of 'self.idx'
        pred[X_flat[:, self.iii['dim']] < self.thresh] = -1
        return pred

    def __calculate_loss(self,y, y_pred):
        loss = (1/len(y)) * np.sum(np.square(y-y_pred))
        return loss
def update_residuals(y_train, left_indices, right_indices, left_residuals, right_residuals):
    # residuals = np.zeros_like(y_train, dtype=float)
    # print(y_train[right_indices])
    # print(y_train[left_indices])
    y_train[left_indices] = y_train[left_indices] - left_residuals
    y_train[right_indices] = y_train[right_indices] - right_residuals
    # print(y_train[right_indices])
    # print(y_train[left_indices])
    return y_train

def update_residualss(residuals, left_indices, right_indices, left_mean_residual, right_mean_residual):
    # updated_residuals = residuals.copy()  # Create a copy of residuals to avoid modifying the original array
    residuals[left_indices] -= left_mean_residual
    residuals[right_indices] -= right_mean_residual
    return residuals

In [ ]:
mse_values = []
num_trees = []
best_model = None
best_mse = float('inf')
lambd = 0.01
ssr = []
stumps = []
for i in range(NumOfStumps):
    print(i)
    st = DecisionStump()
    # print(residue)
    st.train(X, residue)
    k = st.iii
    # print(k)
    # print(lambd * k["right_mean"])
    left_indices = X[:, k["dim"]] <= k["split"]
    right_indices = ~left_indices
    left_indexes = np.where(left_indices)[0]
    right_indexes = np.where(right_indices)[0]

    # predi = np.zeros(n)
    # np.put(predi, left_idx, np.repeat(np.mean(yi[left_idx]), r))  # replace left side mean y
    # np.put(predi, right_idx, np.repeat(np.mean(yi[right_idx]), n-r))  # right side mean y
    
    # predi = predi[:,None]  # make long vector (nx1) in compatible with y
    # predf = predf + predi  # final prediction will be previous prediction value + new prediction of residual
    
    # ei = y - predf  # needed originl y here as residual always from original y    
    # yi = ei # update yi as residual to reloop
    stumps.append(st)
    # print(right_indices)
    # print(left_indexes)
    # print(len(right_labels))

    update_residualss(residue, left_indexes, right_indexes, lambd * k["left_mean"], lambd * k["right_mean"])
    ssr.append(k["ssr"])
    mse_values.append(np.sum(np.square(residue))/len(X))
    print("MSE::",mse_values[-1])

In [ ]:
plt.plot(range(NumOfStumps), mse_values, marker='o', linestyle='-')
plt.xlabel('Index of Stumps')
plt.ylabel('MSE Values')
plt.title('MSE Values vs. Index of Stumps')
plt.grid(True)
plt.show()
best_model_stumps = stumps[:NumOfStumps]
test_pred = np.zeros(len(x_real_test))
maxx = 100
for s in best_model_stumps:
    test_pred += 0.01 * s.predict(pcaa2)
    test_mse = np.mean((y_real_test - test_pred) ** 2)
    maxx = min(test_mse, maxx)

print("Best Model Test MSE:", maxx)